In [ ]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime, date
import time
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm
import App.common as cm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, startstopFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2, figures)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, interact_manual, interactive, IntSlider, Output
from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, Label, Text, Span, HoverTool #, Range1d#, LabelSet
from bokeh.plotting import figure
output_notebook(hide_banner=True)

# login to myplant()
cred()
mp = MyPlant(0)
#Engine._list_cached_validations()
#mp._fetch_installed_base(); # refresh local installed fleet database

In [ ]:
def sfun(x):
    #return all([ ("Flughafen Köln-Bonn" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("CREYKE BECK" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Yukon Energy" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    return all([ ("Forsa Hartmoor" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("EWB" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Pforzheim" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("BMW Landshut 4.10" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T

In [ ]:
motor = fleet.iloc[0]
modes = ['undefined','OFF','MAN','AUTO']
success = ['success','failed','undefined']
e=Engine.from_fleet(mp,motor)
#pp_from='2022-03-28 06:00' # 1 Start
#pp_to='2022-03-28 08:14'
pp_from='2023-05-24'
pp_to='2023-06-04'
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

In [ ]:
cm.dfigures(e)

In [ ]:
fsm = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm.run0(enforce=True, silent=False, debug=False)
fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm.run2(silent=False)
fsm.run4(silent=False)

In [ ]:
print(f"FSM  {fsm.results['starts_counter']} {fsm.results['first_message']} {fsm.results['last_message']}")

In [ ]:
rdf = fsm.starts
startversuch = rdf.iloc[0]
pre_period=5*60
post_period=21*60
t_range=(0,100)
print(f"Start: {startversuch['starttime']} End:{startversuch['endtime']}")
tns = pd.to_datetime((startversuch['starttime'].timestamp() - pre_period + t_range[0]/100.0 * ((startversuch['endtime'].timestamp()  - startversuch['starttime'].timestamp()) + pre_period + post_period)), unit='s')
tne = pd.to_datetime((startversuch['starttime'].timestamp() - pre_period + t_range[1]/100.0 * ((startversuch['endtime'].timestamp()  - startversuch['starttime'].timestamp()) + pre_period + post_period)), unit='s')
print(f"Start: {tns} End:{tne}")

In [ ]:
startversuch['starttime'].timestamp(), startversuch['endtime'].timestamp(), startversuch['endtime'] - startversuch['starttime'], tns, tne

In [ ]:
pre_period, post_period, t_range

In [ ]:
(startversuch['endtime']  - startversuch['starttime']).seconds